<h1 style="background-color: #80a5d6; font-family:Quicksand; font-weight: 500; color:white;line-height: 3em; padding-left: 1em; "> KHÁM PHÁ MỐI QUAN HỆ TRONG DỮ LIỆU </h1>

<h4> 📂 Import các thư viện cần thiết </h4>

In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import plotly.express as px

# remove warnings
import warnings
warnings.simplefilter('ignore')

<h4> 📂 Đọc dữ liệu và in ra 5 dòng đầu tiên </h4>

In [2]:
data = pd.read_excel('../datasets/global_superstore_clean.xlsx', index_col = 0).reset_index()
data.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,1,MX-2015-SC2057582-42279,2015-10-02,2015-10-06,Standard Class,SC-2057582,Sonia Cooley,Consumer,Mexico City,Distrito Federal,...,OFF-LA-4658,Office Supplies,Labels,"Hon File Folder Labels, Adjustable",13.08,3,0.0,4.56,2.033,Medium
1,2,MX-2013-KW1657028-41562,2013-10-15,2013-10-20,Standard Class,KW-1657028,Kelly Williams,Consumer,Dos Quebradas,Risaralda,...,FUR-FU-6238,Furniture,Furnishings,"Tenex Clock, Durable",252.16,8,0.0,90.72,13.449,Medium
2,3,MX-2013-KW1657028-41562,2013-10-15,2013-10-20,Standard Class,KW-1657028,Kelly Williams,Consumer,Dos Quebradas,Risaralda,...,FUR-BO-4845,Furniture,Bookcases,"Ikea 3-Shelf Cabinet, Mobile",193.28,2,0.0,54.08,9.627,Medium
3,4,MX-2013-KW1657028-41562,2013-10-15,2013-10-20,Standard Class,KW-1657028,Kelly Williams,Consumer,Dos Quebradas,Risaralda,...,OFF-BI-3720,Office Supplies,Binders,"Cardinal Binder, Clear",35.44,4,0.0,4.96,1.371,Medium
4,5,MX-2013-KW1657028-41562,2013-10-15,2013-10-20,Standard Class,KW-1657028,Kelly Williams,Consumer,Dos Quebradas,Risaralda,...,OFF-AR-5905,Office Supplies,Art,"Sanford Canvas, Water Color",71.60,2,0.0,11.44,3.787,Medium


---

<h3 style="background-color: #80a5d6; font-family: Quicksand; font-weight: 500; color:white; padding: 1em; line-height: 1.5em; "> Các sản phẩm nào trong Superstore thường được khách hàng mua cùng nhau? </h3>

🔥 **Lợi ích khi trả lời câu hỏi:** Khi các doanh nghiệp, quản lý cửa hàng và nhà đầu tư của Superstore biết được các sản phẩm thường được khách hàng mua cùng nhau, họ có thể áp dụng phương pháp gợi ý sản phẩm *(recommendation system)* để tăng cường trải nghiệm mua hàng của khách hàng và tăng doanh số bán hàng cho doanh nghiệp. Cụ thể, khi khách hàng chọn mua một sản phẩm, hệ thống sẽ gợi ý cho khách hàng các sản phẩm khác thường được mua cùng sản phẩm đó. Điều này giúp khách hàng tìm được các sản phẩm liên quan và tương tự nhau, cải thiện trải nghiệm mua sắm của khách hàng, đồng thời cũng giúp doanh nghiệp tăng doanh số và tính hiệu quả kinh doanh. Ngoài ra, việc biết được các sản phẩm thường được mua cùng nhau cũng giúp doanh nghiệp lên kế hoạch sản xuất và kinh doanh hiệu quả hơn, đồng thời tối ưu hóa quy trình đặt hàng, lưu kho và vận chuyển hàng hóa.

🔥 **Lý thuyết:** Ta sử dụng thư viện `mlxtend` của Python để giải quyết bài toán trên. Thư viện cung cấp thuật toán **Apriori** khai thác tập phổ biến trong dữ liệu và hàm `association_rules` tìm luật kết hợp thỏa mãn `min_sup` và `min_confidence` cho trước.

- **Min_sup (minimum support threshold):** là ngưỡng tối thiểu của sự xuất hiện chung giữa các mục trong tập dữ liệu. Nó được định nghĩa là số lần xuất hiện của một itemset chia cho tổng số mẫu trong cơ sở dữ liệu. Nếu một itemset không đáp ứng `min_sup`, nó sẽ không được xem xét trong quá trình tìm kiếm các luật kết hợp. Giá trị `min_sup` được sử dụng để giảm số lượng các luật được tìm thấy trong quá trình khai thác và giúp chúng ta tìm ra các itemsets có tính kết hợp cao hơn. Ở đây ta chọn $min\_sup = 0.02$.

- **Min_confidence (minimum confidence threshold):** là ngưỡng tối thiểu của độ tin cậy của luật kết hợp. Nó được định nghĩa là xác suất của các item được mua cùng nhau trong một giao dịch có chứa item đó. Nếu độ tin cậy của một luật kết hợp không đáp ứng `min_confidence`, luật này không được coi là có giá trị. Giá trị `min_confidence` được sử dụng để lọc các luật kết hợp có tính kết hợp không chắc chắn và giúp chúng ta tìm ra các luật kết hợp có giá trị thực tế trong dữ liệu. Ở đây ta chọn $min\_confidence = 0.24$.

🔥 **Áp dụng:** Bài toán đặt ra cho thuật toán **Apriori** gắn liền với 2 thuật ngữ: *itemsets* (danh sách sản phẩm) và *transactions* (giao dịch). 

- Đối với tập dữ liệu Superstore, dữ liệu trong hai cột `Sub-Category` hoặc `Product Name` có thể được coi là một itemsets. Tuy nhiên, nếu chọn sản phẩm từ cột `Product-Name` thì tên sản phẩm quá dài, đôi chỗ có xen vào mô tả sản phẩm đó nên ta phải thêm công đoạn trích xuất tên sản phẩm (ngắn gọn) phục vụ mục đích trực quan hóa. Bên cạnh đó, số lượng sản phẩm rất lớn với 3788 sản phẩm, dữ liệu với hơn 51000 dòng nên số tập phổ biến tìm được là không nhiều. Do đó, nhóm quyết định chọn cột `Sub-Category` làm itemsets.

- Nhóm chọn một đơn hàng ứng với một transactions. Các dòng có cùng `Order ID` sẽ được tính là một đơn hàng. Theo quan sát, các dòng có cùng `Order ID` có thể không có cùng `Order Date` (ngày đặt hàng) nhưng lại có chung `Ship Date` (ngày giao hàng) và `Customer ID` (mã khách hàng). Như vậy ta có thể suy đoán rằng Superstore gom các lần đặt hàng của khách hàng lại với nhau để giao hàng một lần cho thuận tiện. Vì thế, ta có thể xem một lần giao hàng như vậy (gom các dòng có cùng `Order ID`) là một đơn hàng.

🔥 **Các bước thực hiện:**

- **Bước 1:** Trích xuất dữ liệu theo từng đơn hàng.
- **Bước 2:** Áp dụng thuật toán **Apriori** khai thác tập phổ biến.
- **Bước 3:** Khai thác luật kết hợp.

---

<font size="5"> 🛒 <strong>Bước 1:</strong> Trích xuất dữ liệu theo từng đơn hàng. </font>

In [3]:
# Group by theo Order_id
df_test = data.groupby(['Order ID'])

dataset = []
for k, d in df_test:
    tmp = list(set(d['Sub-Category']))
    # print(tmp)
    dataset.append(tmp)

# In ra tập các itemsets được mua trong các đơn hàng khác nhau
dataset

[['Machines', 'Storage'],
 ['Bookcases', 'Fasteners'],
 ['Storage', 'Supplies'],
 ['Phones', 'Storage', 'Art', 'Binders'],
 ['Labels'],
 ['Storage'],
 ['Chairs'],
 ['Art'],
 ['Phones'],
 ['Envelopes'],
 ['Binders'],
 ['Fasteners'],
 ['Bookcases'],
 ['Storage', 'Art', 'Fasteners'],
 ['Furnishings'],
 ['Binders'],
 ['Binders', 'Paper'],
 ['Machines'],
 ['Furnishings', 'Appliances', 'Copiers'],
 ['Paper'],
 ['Fasteners'],
 ['Chairs'],
 ['Bookcases'],
 ['Machines', 'Storage', 'Labels', 'Paper', 'Fasteners'],
 ['Labels'],
 ['Storage'],
 ['Appliances'],
 ['Fasteners'],
 ['Furnishings'],
 ['Machines', 'Binders'],
 ['Furnishings', 'Paper'],
 ['Machines',
  'Storage',
  'Bookcases',
  'Phones',
  'Appliances',
  'Art',
  'Binders',
  'Paper'],
 ['Storage', 'Copiers'],
 ['Paper'],
 ['Storage', 'Accessories', 'Binders'],
 ['Bookcases', 'Binders', 'Fasteners'],
 ['Labels', 'Storage', 'Tables', 'Art'],
 ['Machines', 'Fasteners'],
 ['Storage'],
 ['Chairs'],
 ['Envelopes', 'Copiers'],
 ['Art', 'Copie

<font size="5"> 🛒 <strong>Bước 2:</strong> Áp dụng thuật toán **Apriori** khai thác tập phổ biến. </font>

In [4]:
# Encoding
tr = TransactionEncoder()
tr_arr = tr.fit(dataset).transform(dataset)
df = pd.DataFrame(tr_arr, columns = tr.columns_)
df

,Accessories,Appliances,Art,Binders,Bookcases,Chairs,Copiers,Envelopes,Fasteners,Furnishings,Labels,Machines,Paper,Phones,Storage,Supplies,Tables
0,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False
1,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False
3,False,False,True,True,False,False,False,False,False,False,False,False,False,True,True,False,False
4,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25723,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
25724,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
25725,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False
25726,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False


In [5]:
# Áp dụng thuật toán Apriori in ra các tập phổ biến thỏa min_support = 0.02
frequent_itemsets = apriori(df, min_support = 0.02, use_colnames = True)
frequent_itemsets

,support,itemsets
0,0.112446,(Accessories)
1,0.065143,(Appliances)
2,0.170787,(Art)
3,0.211132,(Binders)
4,0.088814,(Bookcases)
5,0.124223,(Chairs)
6,0.082556,(Copiers)
7,0.088308,(Envelopes)
8,0.096432,(Fasteners)
9,0.114894,(Furnishings)


In [6]:
# Đếm số lượng các sản phẩm để trực quan hóa 
item = data.groupby(['Sub-Category']).agg({'Sub-Category': 'count', 'Quantity': 'sum', 'Discount': 'max', 'Sales': 'mean', 'Profit': 'mean'})

item['Discount'] = item['Discount'].apply(lambda x: "{:.0f}%".format(x * 100))
item['Sales'] = item['Sales'].round(2)
item['Profit'] = item['Profit'].round(2)

item = item.rename(columns = {'Sub-Category': 'Số lượt mua',
                              'Quantity': 'Tổng số đã bán',
                              'Discount': 'Mức giảm giá cao nhất',
                              'Sales': 'Doanh thu trung bình ($)',
                              'Profit': 'Lợi nhuận trung bình ($)'})

item.reset_index(inplace = True)
item.sort_values('Số lượt mua', ascending=False)

,Sub-Category,Số lượt mua,Tổng số đã bán,Mức giảm giá cao nhất,Doanh thu trung bình ($),Lợi nhuận trung bình ($)
3,Binders,6146,21403,80%,75.15,11.79
14,Storage,5049,16884,70%,223.18,21.47
2,Art,4864,16215,70%,76.40,11.89
12,Paper,3492,12672,70%,69.24,16.64
5,Chairs,3434,12336,70%,437.30,40.88
13,Phones,3357,11870,70%,508.44,64.56
9,Furnishings,3154,11163,80%,122.12,14.85
0,Accessories,3075,10946,70%,243.65,42.15
10,Labels,2601,9301,70%,28.20,5.76
8,Fasteners,2601,9051,80%,34.41,5.32


In [7]:
fig = px.treemap(item, 
                 path = ['Sub-Category'], 
                 values = 'Số lượt mua', 
                 hover_data = ['Tổng số đã bán', 'Mức giảm giá cao nhất', 'Doanh thu trung bình ($)', 'Lợi nhuận trung bình ($)'],
                 title = 'So sánh các sản phẩm theo số lượt mua hàng từ 2014-2018')

fig.show()

### 🔥 Lý do lựa chọn biểu đồ Treemap:

- Treemap cho phép hiển thị tỷ lệ giữa các thành phần một cách rõ ràng, làm cho việc so sánh và phân tích dữ liệu dễ dàng hơn. Cụ thể, ta dễ dàng so sánh sản phẩm nào được mua nhiều nhất, ít nhất thông qua kích thước của chúng.

- Treemap cho phép nhìn thấy cấu trúc của dữ liệu phân cấp một cách dễ dàng, nhìn vào người dùng sẽ thấy ngay ta đang so sánh số lượng sản phẩm được mua tại Superstore. Nói cách khác, treemap cung cấp cái nhìn tổng quan về dữ liệu một cách nhanh chóng.

- Treemap có khả năng hiển thị màu sắc để tô đậm giá trị cao hơn (ứng với các sản phẩm được mua nhiều sẽ được vẽ với kích thước lớn và màu sắc đậm), giúp tăng tính trực quan và thu hút sự chú ý của người xem.

- Treemap hiển thị các thông tin chi tiết khác về mỗi đối tượng thông qua labels. Ở đây khi người dùng quan tâm đến một sản phẩm nào đó, họ có thể đọc các thông tin chi tiết khác ở labels như số lần mua, mức giảm giá cao nhất, doanh thu và lợi nhuận trung bình.

### 🔥 Nhận xét:

- Qua biểu đồ Treemap, ta có thể thấy Binders là sản phẩm có số lượt mua nhiều nhất (6146 lượt mua) với mức giá ưu đãi cao, lên tới 80%. Theo sau đó là Storage với số lượt mua là 5049, với mức ưu đãi cao lên tới có 70%. Tables có số lượt mua ít nhất (chỉ với 861 lượt và số lượng bán là 3083) có doanh thu lớn nhất(879.26$) tuy nhiên lợi nhuận lại là âm. Binders có doanh thu nhiều nhưng lợi nhuận khá ít, có thể do mức giảm giá quá cao. Ngược lại, copiers (máy photo) tuy ít được mua (với 2223 lượt) trong vòng 4 năm nhưng đem lại lợi nhuận trung bình cao nhất: 116.31$.

- Qua thuật toán **Apriori**, ta thấy rằng có nhiều sản phẩm được mua cùng nhau như: 
    + Binders và Accessories
    + Binders và Art
    + Storage và Art
    + Binders và Paper	
    + Binders và Chairs	
    + Binders và Storage	
    + ...

$\rightarrow$ Các chủ cửa hàng cần chú ý bố trí các sản phẩm này gần nhau để khách hàng thuận tiện trong khâu mua hàng hơn, hoặc tăng cường các chiến dịch Marketing như "Mua 1 tặng 1": tặng kèm hoặc giảm giá một sản phẩm khi mua cùng một sản phẩm khác...

<font size="5"> 🛒 <strong>Bước 3:</strong> Khai thác luật kết hợp. </font>

In [8]:
rules = association_rules(frequent_itemsets, metric = "confidence", min_threshold = 0.1)
print('Số luật kết hợp thu được là:', rules.shape[0], 'luật.')

Số luật kết hợp thu được là: 16 luật.


In [9]:

rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Accessories),(Binders),0.112446,0.211132,0.021883,0.194608,0.921735,-0.001858,0.979483,-0.087314
1,(Binders),(Accessories),0.211132,0.112446,0.021883,0.103645,0.921735,-0.001858,0.990182,-0.097176
2,(Art),(Binders),0.170787,0.211132,0.032299,0.189122,0.895751,-0.003759,0.972856,-0.123078
3,(Binders),(Art),0.211132,0.170787,0.032299,0.152982,0.895751,-0.003759,0.978980,-0.128563
4,(Storage),(Art),0.177433,0.170787,0.030356,0.171084,1.001743,0.000053,1.000359,0.002115
5,(Art),(Storage),0.170787,0.177433,0.030356,0.177742,1.001743,0.000053,1.000376,0.002098
6,(Chairs),(Binders),0.124223,0.211132,0.022427,0.180538,0.855097,-0.003800,0.962666,-0.162124
7,(Binders),(Chairs),0.211132,0.124223,0.022427,0.106222,0.855097,-0.003800,0.979860,-0.176827
8,(Furnishings),(Binders),0.114894,0.211132,0.022155,0.192828,0.913307,-0.002103,0.977324,-0.096857
9,(Binders),(Furnishings),0.211132,0.114894,0.022155,0.104934,0.913307,-0.002103,0.988872,-0.107404


In [10]:
fig = px.scatter(rules['support'], rules['confidence'])
fig.update_layout(
    xaxis_title = "support",
    yaxis_title = "confidence",
    title = ('Support vs Confidence')
)

fig.show()

### 🔥 Lý do lựa chọn biểu đồ Scatter:

- Scatter cho phép quan sát sự phân bố của các điểm dữ liệu trên mặt phẳng dựa theo *support* và *confidence* của nó, ở đây mỗi điểm ứng với một luật kết hợp. Từ đó giúp ta có thể đánh giá sự phân bố của các luật này và quan tâm phân bố của các luật có *support*, *confidence* cao.

- Scatter giúp phát hiện các điểm ngoại lai (outliers). Ta có thể lựa chọn bỏ qua các luật có *support* cao, *confidence* thấp và ngược lại.

### 🔥 Nhận xét:

- Qua biểu đồ Scatter, các luật kết hợp phân bố thưa thớt, chủ yếu tập trung ở mức $0.16 < min\_sup < 0.2$ và $2 < min\_conf < 14$. Có duy nhất một luật có $min\_sup > 0.2$, đạt $min\_sup$ lớn nhất và $min\_conf < 12$. 

- Ta rút trích được 16 luật kết hợp. Có thể kể đến vài luật tiêu biểu như:

    + Paper $\rightarrow$ Binders $(sup = 0.025653, conf = 0.206314)$: Nếu mua giấy sẽ mua binders(tập bìa còng chứa tài liệu). (1)
    + Storage $\rightarrow$ Binders $(sup = 0.034787, conf = 0.196057)$: Nếu mua các vật dụng chứa đồ (tủ, giá sách, ngăn kéo...) sẽ mua binders. (2)
    + Accessories $\rightarrow$ Binders $(sup = 0.021883, conf = 0.194608)$: Nếu mua phụ kiện công nghệ sẽ mua binders. (3)
    + Phone $\rightarrow$ Binders $(sup = 0.023710, conf = 0.194206)$: Nếu mua điện thoại sẽ mua binders. (4)

$\rightarrow$ Luật (1) khá hợp lý theo logic thông thường, tuy nhiên luật (2), (3) và (4) cần kiểm chứng thêm. Các cửa hàng cần cân nhắc lựa chọn một vài luật nêu trên cho chiến lược kinh doanh của mình.

<h4 style = "margin-bottom: 0; "> 📂 References: </h4>

- [Anubha Singh: Groceries Asscociation Rule (Apriori-Algorithm)](https://www.kaggle.com/code/cerolacia/groceries-asscociation-rule-apriori-algorithm)